# Tutorial for Lean 4 Jupyter Kernel

## Commands

### Running commands in sequence

1. You can run commands like defining a function:

In [1]:
def f := 2

def f := 2
--% env 0

Raw input:
{"cmd": "def f := 2"}
Raw output:
{"env": 0}

The inputs are echoed using [alectryon](https://github.com/cpitclaudel/alectryon?tab=readme-ov-file#as-a-library) style annotation of output messages (info, 🟨warning, ❌error) on the corresponding lines (not columns yet), with details of raw input/outputs to [repl](https://github.com/leanprover-community/repl).

There's a state that looks like `--% env 0`, it is the state after executing the last command, and will be used as the initial state for executing the next command.

You can inspect the raw input/output sent to [repl](https://github.com/leanprover-community/repl) by clicking-to-expand.

2. You can also run commands like inspecting declarations:

In [2]:
#check f

#check f
──────▶  f : Nat
--% env 1

Raw input:
{"cmd": "#check f", "env": 0}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 1, "column": 0},
   "endPos": {"line": 1, "column": 6},
   "data": "f : Nat"}],
 "env": 1}

3. Or run multiple commands across multiple lines:

In [3]:
def add (x y : Nat) : Nat := x + y
#print add

def add (x y : Nat) : Nat := x + y
#print add
──────▶  def add : Nat → Nat → Nat :=
fun x y => x + y
--% env 2

Raw input:
{"cmd": "def add (x y : Nat) : Nat := x + y\n#print add", "env": 1}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 2, "column": 0},
   "endPos": {"line": 2, "column": 6},
   "data": "def add : Nat → Nat → Nat :=\nfun x y => x + y"}],
 "env": 2}

4. See below for an error:

In [4]:
#check g

#check g
       ─▶ ❌ unknown identifier 'g'
--% env 3

Raw input:
{"cmd": "#check g", "env": 2}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 1, "column": 7},
   "endPos": {"line": 1, "column": 8},
   "data": "unknown identifier 'g'"}],
 "env": 3}

### Backtracking

Begin with a state like `%env 0` (the syntax mimics Jupyter magic but the output will be prepended with `--` so they are valid Lean comments), you can backtrack. You can actually enter `--%env 0`, `% env 0` or `%e0`. we are trying to be flexible here so you don't need to worry about invalid Lean code, spaces and exact spelling.

The following goes back before declaring `add` and try to inspect `add`, which will result in an error:

In [5]:
%env 1
#check add

--%env 1
#check add
       ───▶ ❌ unknown identifier 'add'
--% env 4

Raw input:
{"cmd": "--%env 1\n#check add", "env": 1}
Raw output:
{"messages":
 [{"severity": "error",
   "pos": {"line": 2, "column": 7},
   "endPos": {"line": 2, "column": 10},
   "data": "unknown identifier 'add'"}],
 "env": 4}

If you enter [Built-in magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html), this should not cause an error, but also has no effect:

In [6]:
%history
def h := 3

--%history
def h := 3
--% env 5

Raw input:
{"cmd": "--%history\ndef h := 3", "env": 4}
Raw output:
{"env": 5}

## Proofs

### Proving sorries

You can declare a theorem to prove with `sorry`

In [7]:
theorem thm1 : 1 = 1 := sorry

theorem thm1 : 1 = 1 := sorry
        ────▶ 🟨 declaration uses 'sorry'
--% env 6
--% prove 0

Raw input:
{"cmd": "theorem thm1 : 1 = 1 := sorry", "env": 5}
Raw output:
{"sorries":
 [{"proofState": 0,
   "pos": {"line": 1, "column": 24},
   "goal": "⊢ 1 = 1",
   "endPos": {"line": 1, "column": 29}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 1, "column": 8},
   "endPos": {"line": 1, "column": 12},
   "data": "declaration uses 'sorry'"}],
 "env": 6}

By default, what you input next would still be another command:

In [8]:
#check thm1

#check thm1
──────▶  thm1 : 1 = 1
--% env 7

Raw input:
{"cmd": "#check thm1", "env": 6}
Raw output:
{"messages":
 [{"severity": "info",
   "pos": {"line": 1, "column": 0},
   "endPos": {"line": 1, "column": 6},
   "data": "thm1 : 1 = 1"}],
 "env": 7}

Begin with `%proof` will get you into tactic mode to proof the theorem immediately before. Note that you can actually enter `--%proof`, `% proof`, `%prove` or `%p`.

In [9]:
theorem thm2 : 2 = 2 := sorry

theorem thm2 : 2 = 2 := sorry
        ────▶ 🟨 declaration uses 'sorry'
--% env 8
--% prove 1

Raw input:
{"cmd": "theorem thm2 : 2 = 2 := sorry", "env": 7}
Raw output:
{"sorries":
 [{"proofState": 1,
   "pos": {"line": 1, "column": 24},
   "goal": "⊢ 2 = 2",
   "endPos": {"line": 1, "column": 29}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 1, "column": 8},
   "endPos": {"line": 1, "column": 12},
   "data": "declaration uses 'sorry'"}],
 "env": 8}

In [10]:
% proof
rfl

--% proof
rfl
▶  Goals accomplished! 🐙
--% env 8
--% prove 2

Raw input:
{"tactic": "--% proof\nrfl", "proofState": 1}
Raw output:
{"proofState": 2, "goals": []}

Great!  Goals accomplished! 🐙 

In [11]:
theorem thm3 {x y : Nat} : x + y = y + x := sorry

theorem thm3 {x y : Nat} : x + y = y + x := sorry
        ────▶ 🟨 declaration uses 'sorry'
--% env 9
--% prove 3

Raw input:
{"cmd": "theorem thm3 {x y : Nat} : x + y = y + x := sorry", "env": 8}
Raw output:
{"sorries":
 [{"proofState": 3,
   "pos": {"line": 1, "column": 44},
   "goal": "x y : Nat\n⊢ x + y = y + x",
   "endPos": {"line": 1, "column": 49}}],
 "messages":
 [{"severity": "warning",
   "pos": {"line": 1, "column": 8},
   "endPos": {"line": 1, "column": 12},
   "data": "declaration uses 'sorry'"}],
 "env": 9}

You can also backtrack to a proof state for the goal you want to prove like `--% prove 7`.  Note that you can actually enter just `%prove 3`, `%proof 3` or `%p3`.

In [12]:
% prove 3
rfl

--% prove 3
rfl
▶ ❌ Lean error:
The rfl tactic failed. Possible reasons:
- The goal is not a reflexive relation (neither `=` nor a relation with a @[refl] lemma).
- The arguments of the relation are not equal.
Try using the reflexivitiy lemma for your relation explicitly, e.g. `exact Eq.rfl`.
x y : Nat
⊢ x + y = y + x
--% env 9

Raw input:
{"tactic": "--% prove 3\nrfl", "proofState": 3}
Raw output:
{"message":
 "Lean error:\nThe rfl tactic failed. Possible reasons:\n- The goal is not a reflexive relation (neither `=` nor a relation with a @[refl] lemma).\n- The arguments of the relation are not equal.\nTry using the reflexivitiy lemma for your relation explicitly, e.g. `exact Eq.rfl`.\nx y : Nat\n⊢ x + y = y + x"}

OK, `rfl` is not enough, let's try some hint:

In [13]:
% prove 3
exact?

--% prove 3
exact?
▶  Try this: exact Nat.add_comm x y
▶  Goals accomplished! 🐙
--% env 9
--% prove 4

Raw input:
{"tactic": "--% prove 3\nexact?", "proofState": 3}
Raw output:
{"proofState": 4,
 "messages":
 [{"severity": "info",
   "pos": {"line": 0, "column": 0},
   "endPos": {"line": 0, "column": 0},
   "data": "Try this: exact Nat.add_comm x y"}],
 "goals": []}

Let's use the hint:

In [14]:
% prove 3
exact Nat.add_comm x y

--% prove 3
exact Nat.add_comm x y
▶  Goals accomplished! 🐙
--% env 9
--% prove 5

Raw input:
{"tactic": "--% prove 3\nexact Nat.add_comm x y", "proofState": 3}
Raw output:
{"proofState": 5, "goals": []}

Voilà! That's what we have so far.

Next, you might want to check out [`examples/demo_proj/demo.ipynb`](https://nbviewer.org/github/utensil/lean4_jupyter/blob/main/examples/demo_proj/demo.ipynb?flush_cache=true) for use Lean 4 Jupyter kernel in a project.